# Test accuracy of LLM Transition Model

In [4]:
#enable autoreload
%load_ext autoreload
%autoreload 2

In [30]:
import sys

sys.path.append('..')

from environments.ElevatorEnvironment import ElevatorEnvironment
from agents.llmzero import LLMTransitionModel
from agents.random_agent import RandomAgent
from agents.elevator_expert import ElevatorExpertPolicyAgent

import numpy as np
import random
import matplotlib.pyplot as plt

Load default elevator environment

In [18]:
env = ElevatorEnvironment()

c:\Users\ianch\miniconda3\envs\aiplanning\Lib\site-packages\pyRDDLGym\Examples c:\Users\ianch\miniconda3\envs\aiplanning\Lib\site-packages\pyRDDLGym\Examples\manifest.csv
Available example environment(s):
CartPole_continuous -> A simple continuous state-action MDP for the classical cart-pole system by Rich Sutton, with actions that describe the continuous force applied to the cart.
CartPole_discrete -> A simple continuous state MDP for the classical cart-pole system by Rich Sutton, with discrete actions that apply a constant force on either the left or right side of the cart.
Elevators -> The Elevator domain models evening rush hours when people from different floors in a building want to go down to the bottom floor using elevators.
HVAC -> Multi-zone and multi-heater HVAC control problem
MarsRover -> Multi Rover Navigation, where a group of agent needs to harvest mineral.
MountainCar -> A simple continuous MDP for the classical mountain car control problem.
NewLanguage -> Example with

c:\Users\ianch\miniconda3\envs\aiplanning\Lib\site-packages\pyRDDLGym\Core\Env\RDDLConstraints.py:85: UserWarning: Constraint does not have a structure of <action or state fluent> <op> <rhs>, where:
<op> is one of {<=, <, >=, >}
<rhs> is a deterministic function of non-fluents or constants only.
>> ( sum_{?f: floor} [ elevator-at-floor(?e, ?f) ] ) == 1
  warnings.warn(


Define agent to generate trajectories for testing and initialize transition model

In [23]:
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

In [31]:
random_agent = RandomAgent(env, seed=SEED)
expert_agent = ElevatorExpertPolicyAgent()

params = {
    "env_params": {
        "system_prompt_path": "../prompts/prompt_elevator_transition.txt",
        "extract_state_regex": r"next state:(.*?)```",
        "extract_regex_fallback": [r"next state:(.*)"],
    },
    "load_prompt_buffer_path": None, # update this path to the path of the saved prompt buffer   
}

transition_model = LLMTransitionModel(**params, prompt_buffer_prefix="elevator_transition", debug=True)

Generate trajectories for both agents

In [25]:
# random agent
state, _ = env.reset(SEED)
done = False

random_agent_trajectory = []

while not done:
    action = random_agent.act(state)
    next_state, reward, done, _, _ = env.step(action)
    random_agent_trajectory.append((state, action, reward, next_state, done))
    state = next_state
    
# expert agent
state, _ = env.reset(SEED + 1)  # use a different seed for the expert agent
done = False

expert_agent_trajectory = []

while not done:
    action = expert_agent.act(state)
    next_state, reward, done, _, _ = env.step(action)
    expert_agent_trajectory.append((state, action, reward, next_state, done))
    state = next_state

In [26]:
trajectories_combined = random_agent_trajectory + expert_agent_trajectory

In [33]:
# test the transition model
trajectory = random.choice(trajectories_combined)
state_test, action, next_state_test = trajectory[0], trajectory[1], trajectory[3]
state_text, action_text, next_state_text = env.state_to_text(state_test), env.action_to_text(action), env.state_to_text(next_state_test)

print("State text:\n", state_text)
print("Action text:", action_text)
print("Next state text:\n", next_state_text)

State text:
 People waiting at floor 2: 3
People waiting at floor 3: 3
People waiting at floor 4: 1
People waiting at floor 5: 3
Elevator at floor 3.
There are 4 people in the elevator.
Elevator is moving down.
Elevator door is closed.

Action text: move
Next state text:
 People waiting at floor 2: 3
People waiting at floor 3: 3
People waiting at floor 4: 1
People waiting at floor 5: 3
Elevator at floor 2.
There are 4 people in the elevator.
Elevator is moving down.
Elevator door is closed.



In [34]:
predicted_next_state_text, status = transition_model.get_next_state(state_text, action_text)

predicted_next_state_text

AuthenticationError: Error code: 401 - {'message': 'Unauthorized', 'request_id': 'da749f2bf9b7e049104ce66ee6ab896b'}